In [1]:
# SemChunk + Cohere Multilingual + Qdrant Entegrasyon
# Yargıtay Kararları için Semantic Chunking Pipeline

import pandas as pd
import tiktoken
import semchunk
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import cohere
import numpy as np
import uuid
from typing import List, Dict, Any
import os
from dataclasses import dataclass
import json
from dotenv import load_dotenv

print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))

cohere_api_key=os.getenv("COHERE_API_KEY")


@dataclass
class Config:
    # Cohere ayarları
    COHERE_API_KEY: str = "your_cohere_api_key_here"  # Cohere API anahtarınız
    COHERE_MODEL: str = "embed-multilingual-v3.0"  # Cohere multilingual model
    
    # SemChunk ayarları
    TOKEN_SIZE: int = 512  # Chunk boyutu (token)
    ENCODING_NAME: str = "cl100k_base"  # Tiktoken encoding
    
    # Qdrant ayarları
    QDRANT_URL: str = "http://localhost:6333"  # Lokal Qdrant
    COLLECTION_NAME: str = "yargitay_semantic_chunks"
    EMBEDDING_DIM: int = 1024  # Cohere multilingual embedding boyutu
    
    # Dosya ayarları
    CSV_FILE: str = "path/to/your/yargitay_data.csv"
    BATCH_SIZE: int = 50

class YargitaySemanticProcessor:
    """Yargıtay kararları için semantic chunking ve vector search"""
    
    def __init__(self, config: Config):
        self.config = config
        
        # SemChunk chunker oluştur
        self.encoding = tiktoken.get_encoding(config.ENCODING_NAME)
        self.chunker = semchunk.chunkerify(self.encoding, config.TOKEN_SIZE)
        
        # Cohere client oluştur
        self.cohere_client = cohere.Client(config.COHERE_API_KEY)
        
        # Qdrant client oluştur
        self.qdrant_client = QdrantClient(url=config.QDRANT_URL)
        
        print(f"✅ SemChunk chunker hazır (Token boyutu: {config.TOKEN_SIZE})")
        print(f"✅ Cohere client hazır ({config.COHERE_MODEL})")
        print(f"✅ Qdrant client hazır ({config.QDRANT_URL})")
    
    def test_cohere_connection(self):
        """Cohere bağlantısını test et"""
        try:
            test_response = self.cohere_client.embed(
                texts=["Bu bir test metnidir"],
                model=self.config.COHERE_MODEL,
                input_type="search_document"
            )
            embedding_dim = len(test_response.embeddings[0])
            print(f"✅ Cohere test başarılı - Embedding boyutu: {embedding_dim}")
            return embedding_dim
        except Exception as e:
            print(f"❌ Cohere bağlantı hatası: {e}")
            return None
    
    def create_qdrant_collection(self, recreate: bool = False):
        """Qdrant koleksiyonu oluştur"""
        collection_name = self.config.COLLECTION_NAME
        
        # Koleksiyon varsa ve recreate True ise sil
        if recreate:
            try:
                self.qdrant_client.delete_collection(collection_name)
                print(f"🗑️ Eski koleksiyon silindi: {collection_name}")
            except:
                pass
        
        # Koleksiyon yoksa oluştur
        try:
            collections = self.qdrant_client.get_collections().collections
            collection_names = [c.name for c in collections]
            
            if collection_name not in collection_names:
                self.qdrant_client.create_collection(
                    collection_name=collection_name,
                    vectors_config=VectorParams(
                        size=self.config.EMBEDDING_DIM,
                        distance=Distance.COSINE
                    )
                )
                print(f"✅ Koleksiyon oluşturuldu: {collection_name}")
            else:
                print(f"ℹ️ Koleksiyon zaten var: {collection_name}")
                
        except Exception as e:
            print(f"❌ Koleksiyon oluşturma hatası: {e}")
            raise
    
    def semantic_chunk_text(self, text: str, metadata: dict = None) -> List[Dict]:
        """Metni semantic olarak chunk'lara böl"""
        if not text or text.strip() == "":
            return []
        
        try:
            # SemChunk ile metni böl
            chunks = self.chunker(text)
            
            result_chunks = []
            for i, rawText in enumerate(chunks):
                if rawText.strip():  # Boş chunk'ları atla
                    chunk_data = {
                        'chunk_id': i,
                        'text': rawText.strip(),
                        'token_count': len(self.encoding.encode(rawText)),
                        'char_count': len(rawText),
                    }
                    
                    # Metadata ekle
                    if metadata:
                        chunk_data.update(metadata)
                    
                    result_chunks.append(chunk_data)
            
            return result_chunks
            
        except Exception as e:
            print(f"❌ Chunking hatası: {e}")
            return []
    
    def create_embeddings(self, texts: List[str], batch_size: int = 32) -> List[List[float]]:
        """Metinleri Cohere ile embedding'e çevir"""
        all_embeddings = []
        
        # Cohere API limitleri için batch processing
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            
            try:
                response = self.cohere_client.embed(
                    texts=batch_texts,
                    model=self.config.COHERE_MODEL,
                    input_type="search_document"  # Dokuman indexleme için
                )
                
                batch_embeddings = response.embeddings
                all_embeddings.extend(batch_embeddings)
                
                print(f"  📊 Embedding oluşturuldu: {i+len(batch_texts)}/{len(texts)}")
                
            except Exception as e:
                print(f"❌ Embedding hatası (batch {i//batch_size + 1}): {e}")
                # Hata durumunda boş embedding ekle
                all_embeddings.extend([[0.0] * self.config.EMBEDDING_DIM] * len(batch_texts))
        
        return all_embeddings
    
    def process_csv_file(self, csv_path: str) -> List[Dict]:
        """CSV dosyasını işle ve chunk'ları oluştur"""
        print(f"📄 CSV dosyası okunuyor: {csv_path}")
        
        try:
            df = pd.read_csv(csv_path)
            print(f"📊 {len(df)} satır veri yüklendi")
        except Exception as e:
            print(f"❌ CSV okuma hatası: {e}")
            return []
        
        # Gerekli sütunları kontrol et
        required_columns = ['chunk_text']  # Ana metin sütunu
        optional_columns = ['esas_no', 'karar_no', 'location', 'dates', 'karar_turu']
        
        if 'rawText' not in df.columns:
            print(f"❌ 'chunk_text' sütunu bulunamadı. Mevcut sütunlar: {df.columns.tolist()}")
            return []
        
        all_chunks = []
        
        print("🔄 Semantic chunking başlıyor...")
        for idx, row in df.iterrows():
            # Ana metni al
            text = row.get('chunk_text', '') or row.get('text', '')
            
            if not text or pd.isna(text):
                continue
            
            # Metadata hazırla
            metadata = {
                'original_index': idx,
                'esas_no': row.get('esas_no', ''),
                'karar_no': row.get('karar_no', ''),
                'daire': row.get('location', ''),
                'tarih': row.get('dates', ''),
                'karar_turu': row.get('karar_turu', ''),
            }
            
            # Semantic chunking yap
            chunks = self.semantic_chunk_text(str(text), metadata)
            all_chunks.extend(chunks)
            
            # Progress göster
            if (idx + 1) % 100 == 0:
                print(f"  ✅ İşlenen satır: {idx + 1}/{len(df)} (Toplam chunk: {len(all_chunks)})")
        
        print(f"🧩 Toplam {len(all_chunks)} chunk oluşturuldu")
        return all_chunks
    
    def upload_to_qdrant(self, chunks: List[Dict]):
        """Chunk'ları Qdrant'a yükle"""
        if not chunks:
            print("❌ Yüklenecek chunk yok")
            return
        
        print(f"🚀 {len(chunks)} chunk Qdrant'a yükleniyor...")
        
        # Metinleri topla
        texts = [chunk['text'] for chunk in chunks]
        
        # Embedding'leri oluştur
        print("🔮 Embedding'ler oluşturuluyor...")
        embeddings = self.create_embeddings(texts)
        
        if len(embeddings) != len(chunks):
            print(f"❌ Embedding sayısı uyumsuz: {len(embeddings)} vs {len(chunks)}")
            return
        
        # Qdrant point'leri hazırla
        points = []
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
            point = PointStruct(
                id=str(uuid.uuid4()),
                vector=embedding,
                payload=chunk
            )
            points.append(point)
        
        # Batch halinde yükle
        batch_size = self.config.BATCH_SIZE
        print(f"📦 {batch_size} batch size ile yükleniyor...")
        
        for i in range(0, len(points), batch_size):
            batch = points[i:i + batch_size]
            
            try:
                self.qdrant_client.upsert(
                    collection_name=self.config.COLLECTION_NAME,
                    points=batch
                )
                print(f"  ✅ Batch yüklendi: {min(i + batch_size, len(points))}/{len(points)}")
                
            except Exception as e:
                print(f"❌ Batch yükleme hatası: {e}")
        
        print("🎉 Yükleme tamamlandı!")
    
    def search_semantic(self, query: str, limit: int = 10, score_threshold: float = 0.7) -> List[Dict]:
        """Semantic arama yap"""
        print(f"🔍 Arama: '{query}'")
        
        try:
            # Query için embedding oluştur
            query_response = self.cohere_client.embed(
                texts=[query],
                model=self.config.COHERE_MODEL,
                input_type="search_query"  # Arama query'si için
            )
            query_embedding = query_response.embeddings[0]
            
            # Qdrant'ta ara
            search_results = self.qdrant_client.search(
                collection_name=self.config.COLLECTION_NAME,
                query_vector=query_embedding,
                limit=limit,
                score_threshold=score_threshold
            )
            
            # Sonuçları formatla
            results = []
            for result in search_results:
                results.append({
                    'score': result.score,
                    'payload': result.payload
                })
            
            print(f"📊 {len(results)} sonuç bulundu")
            return results
            
        except Exception as e:
            print(f"❌ Arama hatası: {e}")
            return []
    
    def get_collection_info(self) -> dict:
        """Koleksiyon bilgilerini al"""
        try:
            info = self.qdrant_client.get_collection(self.config.COLLECTION_NAME)
            return {
                "collection_name": self.config.COLLECTION_NAME,
                "points_count": info.points_count,
                "vectors_count": info.vectors_count,
                "status": info.status
            }
        except Exception as e:
            return {"error": str(e)}

# Ana Pipeline Sınıfı
class YargitayPipeline:
    """Ana pipeline sınıfı"""
    
    def __init__(self, config: Config):
        self.processor = YargitaySemanticProcessor(config)
        self.config = config
    
    def full_pipeline(self, csv_path: str = None):
        """Tam pipeline'ı çalıştır"""
        csv_path = csv_path or self.config.CSV_FILE
        
        print("🚀 Yargıtay Semantic Pipeline Başlıyor")
        print("=" * 50)
        
        # 1. Bağlantıları test et
        embedding_dim = self.processor.test_cohere_connection()
        if not embedding_dim:
            return False
        
        # 2. Koleksiyon oluştur
        self.processor.create_qdrant_collection(recreate=True)
        
        # 3. CSV'yi işle
        chunks = self.processor.process_csv_file(csv_path)
        if not chunks:
            print("❌ İşlenecek chunk bulunamadı")
            return False
        
        # 4. Qdrant'a yükle
        self.processor.upload_to_qdrant(chunks)
        
        # 5. Bilgileri göster
        info = self.processor.get_collection_info()
        print("\n📊 Koleksiyon Bilgileri:")
        print(json.dumps(info, indent=2, ensure_ascii=False))
        
        return True
    
    def interactive_search(self):
        """İnteraktif arama arayüzü"""
        print("\n" + "=" * 50)
        print("🏛️ YARGITAY SEMANTİK ARAMA SİSTEMİ")
        print("=" * 50)
        
        while True:
            query = input("\n🔍 Arama metni (çıkmak için 'q'): ")
            if query.lower() in ['q', 'quit', 'exit']:
                print("👋 Görüşürüz!")
                break
            
            if not query.strip():
                continue
            
            try:
                limit = int(input("📊 Kaç sonuç? (varsayılan 5): ") or "5")
            except:
                limit = 5
            
            # Arama yap
            results = self.processor.search_semantic(query, limit=limit)
            
            if not results:
                print("❌ Sonuç bulunamadı")
                continue
            
            print(f"\n📋 {len(results)} sonuç bulundu:")
            print("-" * 60)
            
            for i, result in enumerate(results, 1):
                payload = result['payload']
                print(f"\n{i}. 📄 Benzerlik Skoru: {result['score']:.3f}")
                print(f"   ⚖️ Esas No: {payload.get('esas_no', 'N/A')}")
                print(f"   📋 Karar No: {payload.get('karar_no', 'N/A')}")
                print(f"   🏛️ Daire: {payload.get('daire', 'N/A')}")
                print(f"   📅 Tarih: {payload.get('tarih', 'N/A')}")
                print(f"   🔤 Token: {payload.get('token_count', 'N/A')}")
                print(f"   📝 Metin Önizleme:")
                
                text = payload.get('text', '')
                preview = text[:300] + "..." if len(text) > 300 else text
                print(f"      {preview}")
                print("-" * 60)

# Kullanım örneği ve main fonksiyon
def main():
    """Ana fonksiyon"""
    
    # Konfigürasyon (buraya kendi bilgilerinizi yazın)
    config = Config(
        COHERE_API_KEY=str(cohere_api_key),  # Cohere API anahtarınız
        CSV_FILE="/home/yapayzeka/ahsen_bulbul/data/new.csv",  # CSV dosya yolunuz
        TOKEN_SIZE=512,  # Chunk boyutu
        QDRANT_URL="http://localhost:6333",  # Lokal Qdrant URL
        COLLECTION_NAME="yargitay_semantic_chunks"
    )
    
    # Pipeline oluştur
    pipeline = YargitayPipeline(config)
    
    # Menü göster
    while True:
        print("\n" + "=" * 50)
        print("🏛️ YARGITAY SEMANTİK CHUNK SİSTEMİ")
        print("=" * 50)
        print("1. Tam pipeline çalıştır (CSV → Semantic Chunks → Qdrant)")
        print("2. İnteraktif arama yap")
        print("3. Koleksiyon bilgilerini göster")
        print("4. Çıkış")
        
        choice = input("\nSeçiminiz (1-4): ")
        
        if choice == "1":
            csv_path = input(f"CSV dosya yolu (Enter: {config.CSV_FILE}): ").strip()
            if not csv_path:
                csv_path = config.CSV_FILE
            
            success = pipeline.full_pipeline(csv_path)
            if success:
                print("✅ Pipeline başarıyla tamamlandı!")
            else:
                print("❌ Pipeline hatası!")
        
        elif choice == "2":
            pipeline.interactive_search()
        
        elif choice == "3":
            info = pipeline.processor.get_collection_info()
            print("\n📊 Koleksiyon Bilgileri:")
            print(json.dumps(info, indent=2, ensure_ascii=False))
        
        elif choice == "4":
            print("👋 Görüşürüz!")
            break
        
        else:
            print("❌ Geçersiz seçim!")

if __name__ == "__main__":
    main()

True
✅ SemChunk chunker hazır (Token boyutu: 512)
✅ Cohere client hazır (embed-multilingual-v3.0)
✅ Qdrant client hazır (http://localhost:6333)

🏛️ YARGITAY SEMANTİK CHUNK SİSTEMİ
1. Tam pipeline çalıştır (CSV → Semantic Chunks → Qdrant)
2. İnteraktif arama yap
3. Koleksiyon bilgilerini göster
4. Çıkış
🚀 Yargıtay Semantic Pipeline Başlıyor
✅ Cohere test başarılı - Embedding boyutu: 1024
🗑️ Eski koleksiyon silindi: yargitay_semantic_chunks
✅ Koleksiyon oluşturuldu: yargitay_semantic_chunks
📄 CSV dosyası okunuyor: /home/yapayzeka/ahsen_bulbul/data/yargitay_cleaned_2025-08-21.csv
📊 17999 satır veri yüklendi
🔄 Semantic chunking başlıyor...
🧩 Toplam 0 chunk oluşturuldu
❌ İşlenecek chunk bulunamadı
❌ Pipeline hatası!

🏛️ YARGITAY SEMANTİK CHUNK SİSTEMİ
1. Tam pipeline çalıştır (CSV → Semantic Chunks → Qdrant)
2. İnteraktif arama yap
3. Koleksiyon bilgilerini göster
4. Çıkış
👋 Görüşürüz!


### version2

In [ ]:
# SemChunk + Cohere Multilingual + Qdrant Entegrasyon
# Yargıtay Kararları için Semantic Chunking Pipeline

import pandas as pd
import tiktoken
import semchunk
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import cohere
import numpy as np
import uuid
from typing import List, Dict, Any
import os
from dataclasses import dataclass
import json
from dotenv import load_dotenv

print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))

cohere_api_key=os.getenv("COHERE_API_KEY")
# Konfigürasyon
@dataclass
class Config:
    # Cohere ayarları
    COHERE_API_KEY: str = cohere_api_key  # Cohere API anahtarınız
    COHERE_MODEL: str = "embed-multilingual-v3.0"  # Cohere multilingual model
    
    # SemChunk ayarları
    TOKEN_SIZE: int = 512  # Chunk boyutu (token)
    ENCODING_NAME: str = "cl100k_base"  # Tiktoken encoding
    
    # Qdrant ayarları
    QDRANT_URL: str = "http://localhost:6333"  # Lokal Qdrant
    COLLECTION_NAME: str = "yargitay_semantic_chunks"
    EMBEDDING_DIM: int = 1024  # Cohere multilingual embedding boyutu
    
    # Dosya ayarları
    CSV_FILE: str = "/home/yapayzeka/ahsen_bulbul/data/10data.csv"
    BATCH_SIZE: int = 16

class YargitaySemanticProcessor:
    """Yargıtay kararları için semantic chunking ve vector search"""
    
    def __init__(self, config: Config):
        self.config = config
        
        # SemChunk chunker oluştur
        self.encoding = tiktoken.get_encoding(config.ENCODING_NAME)
        self.chunker = semchunk.chunkerify(self.encoding, config.TOKEN_SIZE)
        
        # Cohere client oluştur
        self.cohere_client = cohere.Client(config.COHERE_API_KEY)
        
        # Qdrant client oluştur
        self.qdrant_client = QdrantClient(url=config.QDRANT_URL)
        
        print(f"✅ SemChunk chunker hazır (Token boyutu: {config.TOKEN_SIZE})")
        print(f"✅ Cohere client hazır ({config.COHERE_MODEL})")
        print(f"✅ Qdrant client hazır ({config.QDRANT_URL})")
    
    def test_cohere_connection(self):
        """Cohere bağlantısını test et"""
        try:
            test_response = self.cohere_client.embed(
                texts=["Bu bir test metnidir"],
                model=self.config.COHERE_MODEL,
                input_type="search_document"
            )
            embedding_dim = len(test_response.embeddings[0])
            print(f"✅ Cohere test başarılı - Embedding boyutu: {embedding_dim}")
            return embedding_dim
        except Exception as e:
            print(f"❌ Cohere bağlantı hatası: {e}")
            return None
    
    def create_qdrant_collection(self, recreate: bool = False):
        """Qdrant koleksiyonu oluştur"""
        collection_name = self.config.COLLECTION_NAME
        
        # Koleksiyon varsa ve recreate True ise sil
        if recreate:
            try:
                self.qdrant_client.delete_collection(collection_name)
                print(f"🗑️ Eski koleksiyon silindi: {collection_name}")
            except:
                pass
        
        # Koleksiyon yoksa oluştur
        try:
            collections = self.qdrant_client.get_collections().collections
            collection_names = [c.name for c in collections]
            
            if collection_name not in collection_names:
                self.qdrant_client.create_collection(
                    collection_name=collection_name,
                    vectors_config=VectorParams(
                        size=self.config.EMBEDDING_DIM,
                        distance=Distance.COSINE
                    )
                )
                print(f"✅ Koleksiyon oluşturuldu: {collection_name}")
            else:
                print(f"ℹ️ Koleksiyon zaten var: {collection_name}")
                
        except Exception as e:
            print(f"❌ Koleksiyon oluşturma hatası: {e}")
            raise
    
    def semantic_chunk_text(self, text: str, metadata: dict = None) -> List[Dict]:
        """Metni semantic olarak chunk'lara böl"""
        if not text or text.strip() == "":
            return []
        
        try:
            # SemChunk ile metni böl
            chunks = self.chunker(text)
            
            result_chunks = []
            for i, chunk_text in enumerate(chunks):
                if chunk_text.strip():  # Boş chunk'ları atla
                    chunk_data = {
                        'chunk_id': i,
                        'text': chunk_text.strip(),
                        'token_count': len(self.encoding.encode(chunk_text)),
                        'char_count': len(chunk_text),
                    }
                    
                    # Metadata ekle
                    if metadata:
                        chunk_data.update(metadata)
                    
                    result_chunks.append(chunk_data)
            
            return result_chunks
            
        except Exception as e:
            print(f"❌ Chunking hatası: {e}")
            return []
    
    def create_embeddings(self, texts: List[str], batch_size: int = 16) -> List[List[float]]:
        """Metinleri Cohere ile embedding'e çevir"""
        all_embeddings = []
        
        # Cohere API limitleri için batch processing
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            
            try:
                response = self.cohere_client.embed(
                    texts=batch_texts,
                    model=self.config.COHERE_MODEL,
                    input_type="search_document"  # Dokuman indexleme için
                )
                
                batch_embeddings = response.embeddings
                all_embeddings.extend(batch_embeddings)
                
                print(f"  📊 Embedding oluşturuldu: {i+len(batch_texts)}/{len(texts)}")
                
            except Exception as e:
                print(f"❌ Embedding hatası (batch {i//batch_size + 1}): {e}")
                # Hata durumunda boş embedding ekle
                all_embeddings.extend([[0.0] * self.config.EMBEDDING_DIM] * len(batch_texts))
        
        return all_embeddings
    
    def process_csv_file(self, csv_path: str) -> List[Dict]:
        """CSV dosyasını işle ve chunk'ları oluştur"""
        print(f"📄 CSV dosyası okunuyor: {csv_path}")
        
        try:
            df = pd.read_csv(csv_path)
            print(f"📊 {len(df)} satır veri yüklendi")
        except Exception as e:
            print(f"❌ CSV okuma hatası: {e}")
            return []
        
        # Gerekli sütunları kontrol et
        required_columns = ['rawText']  # Ana metin sütunu
        optional_columns = ['esasNo', 'kararNo', 'location', 'extractedDates']
        
        if 'rawText' not in df.columns:
            print(f"❌ 'rawText' sütunu bulunamadı. Mevcut sütunlar: {df.columns.tolist()}")
            return []
        
        all_chunks = []
        
        print("🔄 Semantic chunking başlıyor...")
        for idx, row in df.iterrows():
            # Ana metni al
            text = row.get('rawText', '') or row.get('text', '')
            
            if not text or pd.isna(text):
                continue
            
            # Metadata hazırla
            metadata = {
                'original_index': idx,
                'esas_no': row.get('esasNo', ''),
                'karar_no': row.get('kararNo', ''),
                'daire': row.get('location', ''),
                'tarih': row.get('extractedDates', '')
                #'karar_turu': row.get('karar_turu', ''),
            }
            
            # Semantic chunking yap
            chunks = self.semantic_chunk_text(str(text), metadata)
            all_chunks.extend(chunks)
            
            # Progress göster
            if (idx + 1) % 100 == 0:
                print(f"  ✅ İşlenen satır: {idx + 1}/{len(df)} (Toplam chunk: {len(all_chunks)})")
        
        print(f"🧩 Toplam {len(all_chunks)} chunk oluşturuldu")
        return all_chunks
    
    def upload_to_qdrant(self, chunks: List[Dict]):
        """Chunk'ları Qdrant'a yükle"""
        if not chunks:
            print("❌ Yüklenecek chunk yok")
            return
        
        print(f"🚀 {len(chunks)} chunk Qdrant'a yükleniyor...")
        
        # Metinleri topla
        texts = [chunk['text'] for chunk in chunks]
        
        # Embedding'leri oluştur
        print("🔮 Embedding'ler oluşturuluyor...")
        embeddings = self.create_embeddings(texts)
        
        if len(embeddings) != len(chunks):
            print(f"❌ Embedding sayısı uyumsuz: {len(embeddings)} vs {len(chunks)}")
            return
        
        # Qdrant point'leri hazırla
        points = []
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
            point = PointStruct(
                id=str(uuid.uuid4()),
                vector=embedding,
                payload=chunk
            )
            points.append(point)
        
        # Batch halinde yükle
        batch_size = self.config.BATCH_SIZE
        print(f"📦 {batch_size} batch size ile yükleniyor...")
        
        for i in range(0, len(points), batch_size):
            batch = points[i:i + batch_size]
            
            try:
                self.qdrant_client.upsert(
                    collection_name=self.config.COLLECTION_NAME,
                    points=batch
                )
                print(f"  ✅ Batch yüklendi: {min(i + batch_size, len(points))}/{len(points)}")
                
            except Exception as e:
                print(f"❌ Batch yükleme hatası: {e}")
        
        print("🎉 Yükleme tamamlandı!")
    
    def search_semantic(self, query: str, limit: int = 10, score_threshold: float = 0.7) -> List[Dict]:
        """Semantic arama yap"""
        print(f"🔍 Arama: '{query}'")
        
        try:
            # Query için embedding oluştur
            query_response = self.cohere_client.embed(
                texts=[query],
                model=self.config.COHERE_MODEL,
                input_type="search_query"  # Arama query'si için
            )
            query_embedding = query_response.embeddings[0]
            
            # Qdrant'ta ara
            search_results = self.qdrant_client.search(
                collection_name=self.config.COLLECTION_NAME,
                query_vector=query_embedding,
                limit=limit,
                score_threshold=score_threshold
            )
            
            # Sonuçları formatla
            results = []
            for point in search_results:
                results.append({
                    'score': point.score,
                    'payload': point.payload
                })
            
            print(f"📊 {len(results)} sonuç bulundu")
            return results
            
        except Exception as e:
            print(f"❌ Arama hatası: {e}")
            return []
    
    def get_collection_info(self) -> dict:
        """Koleksiyon bilgilerini al"""
        try:
            info = self.qdrant_client.get_collection(self.config.COLLECTION_NAME)
            return {
                "collection_name": self.config.COLLECTION_NAME,
                "points_count": info.points_count,
                "vectors_count": info.vectors_count,
                "status": info.status
            }
        except Exception as e:
            return {"error": str(e)}

# Ana Pipeline Sınıfı
class YargitayPipeline:
    """Ana pipeline sınıfı"""
    
    def __init__(self, config: Config):
        self.processor = YargitaySemanticProcessor(config)
        self.config = config
    
    def full_pipeline(self, csv_path: str = None):
        """Tam pipeline'ı çalıştır"""
        csv_path = csv_path or self.config.CSV_FILE
        
        print("🚀 Yargıtay Semantic Pipeline Başlıyor")
        print("=" * 50)
        
        # 1. Bağlantıları test et
        embedding_dim = self.processor.test_cohere_connection()
        if not embedding_dim:
            return False
        
        # 2. Koleksiyon oluştur
        self.processor.create_qdrant_collection(recreate=True)
        
        # 3. CSV'yi işle
        chunks = self.processor.process_csv_file(csv_path)
        if not chunks:
            print("❌ İşlenecek chunk bulunamadı")
            return False
        
        # 4. Qdrant'a yükle
        self.processor.upload_to_qdrant(chunks)
        
        # 5. Bilgileri göster
        info = self.processor.get_collection_info()
        print("\n📊 Koleksiyon Bilgileri:")
        print(json.dumps(info, indent=2, ensure_ascii=False))
        
        return True
    
    def interactive_search(self):
        """İnteraktif arama arayüzü"""
        print("\n" + "=" * 50)
        print("🏛️ YARGITAY SEMANTİK ARAMA SİSTEMİ")
        print("=" * 50)
        
        while True:
            query = input("\n🔍 Arama metni (çıkmak için 'q'): ")
            if query.lower() in ['q', 'quit', 'exit']:
                print("👋 Görüşürüz!")
                break
            
            if not query.strip():
                continue
            
            try:
                limit = int(input("📊 Kaç sonuç? (varsayılan 5): ") or "5")
            except:
                limit = 5
            
            # Arama yap
            results = self.processor.search_semantic(query, limit=limit)
            
            if not results:
                print("❌ Sonuç bulunamadı")
                continue
            
            print(f"\n📋 {len(results)} sonuç bulundu:")
            print("-" * 60)
            
            for i, result in enumerate(results, 1):
                payload = result['payload']
                print(f"\n{i}. 📄 Benzerlik Skoru: {result['score']:.3f}")
                print(f"   ⚖️ Esas No: {payload.get('esas_no', 'N/A')}")
                print(f"   📋 Karar No: {payload.get('karar_no', 'N/A')}")
                print(f"   🏛️ Daire: {payload.get('daire', 'N/A')}")
                print(f"   📅 Tarih: {payload.get('tarih', 'N/A')}")
                print(f"   🔤 Token: {payload.get('token_count', 'N/A')}")
                print(f"   📝 Metin Önizleme:")
                
                text = payload.get('text', '')
                preview = text[:300] + "..." if len(text) > 300 else text
                print(f"      {preview}")
                print("-" * 60)

# Kullanım örneği ve main fonksiyon
def main():
    """Ana fonksiyon"""
    
    # Konfigürasyon (buraya kendi bilgilerinizi yazın)
    config = Config(
        COHERE_API_KEY=str(cohere_api_key),  # Cohere API anahtarınız
        CSV_FILE="/home/yapayzeka/ahsen_bulbul/data/10data.csv",  # CSV dosya yolunuz
        TOKEN_SIZE=512,  # Chunk boyutu
        QDRANT_URL="http://localhost:6333",  # Lokal Qdrant URL
        COLLECTION_NAME="yargitay_semantic_chunks"
    )
    
    # Pipeline oluştur
    pipeline = YargitayPipeline(config)
    
    # Menü göster
    while True:
        print("\n" + "=" * 50)
        print("🏛️ YARGITAY SEMANTİK CHUNK SİSTEMİ")
        print("=" * 50)
        print("1. Tam pipeline çalıştır (CSV → Semantic Chunks → Qdrant)")
        print("2. İnteraktif arama yap")
        print("3. Koleksiyon bilgilerini göster")
        print("4. Çıkış")
        
        choice = input("\nSeçiminiz (1-4): ")
        
        if choice == "1":
            csv_path = input(f"CSV dosya yolu (Enter: {config.CSV_FILE}): ").strip()
            if not csv_path:
                csv_path = config.CSV_FILE
            
            success = pipeline.full_pipeline(csv_path)
            if success:
                print("✅ Pipeline başarıyla tamamlandı!")
            else:
                print("❌ Pipeline hatası!")
        
        elif choice == "2":
            pipeline.interactive_search()
        
        elif choice == "3":
            info = pipeline.processor.get_collection_info()
            print("\n📊 Koleksiyon Bilgileri:")
            print(json.dumps(info, indent=2, ensure_ascii=False))
        
        elif choice == "4":
            print("👋 Görüşürüz!")
            break
        
        else:
            print("❌ Geçersiz seçim!")

if __name__ == "__main__":
    main()

True
✅ SemChunk chunker hazır (Token boyutu: 512)
✅ Cohere client hazır (embed-multilingual-v3.0)
✅ Qdrant client hazır (http://localhost:6333)

🏛️ YARGITAY SEMANTİK CHUNK SİSTEMİ
1. Tam pipeline çalıştır (CSV → Semantic Chunks → Qdrant)
2. İnteraktif arama yap
3. Koleksiyon bilgilerini göster
4. Çıkış
🚀 Yargıtay Semantic Pipeline Başlıyor
✅ Cohere test başarılı - Embedding boyutu: 1024
🗑️ Eski koleksiyon silindi: yargitay_semantic_chunks
✅ Koleksiyon oluşturuldu: yargitay_semantic_chunks
📄 CSV dosyası okunuyor: /home/yapayzeka/ahsen_bulbul/data/yargitay_cleaned_2025-08-21.csv
📊 17999 satır veri yüklendi
🔄 Semantic chunking başlıyor...
  ✅ İşlenen satır: 100/17999 (Toplam chunk: 649)
  ✅ İşlenen satır: 200/17999 (Toplam chunk: 1339)
  ✅ İşlenen satır: 300/17999 (Toplam chunk: 1999)
  ✅ İşlenen satır: 400/17999 (Toplam chunk: 2670)
  ✅ İşlenen satır: 500/17999 (Toplam chunk: 3335)
  ✅ İşlenen satır: 600/17999 (Toplam chunk: 4059)
  ✅ İşlenen satır: 700/17999 (Toplam chunk: 4722)
  ✅ İşl

KeyboardInterrupt: 